In [187]:
import gurobipy as gb
from gurobipy import *
import numpy as np
import pandas as pd

In [188]:
dist = pd.read_excel('/Users/arham/Downloads/TSP_Oct_19/TSP.xlsx', sheet_name='Distances', index_col = 0)
dist

,1,2,3,4,5,6,7,8,9
1,0,45,64,25,60,32,30,70,40
2,45,0,30,42,48,25,38,50,26
3,64,30,0,20,50,37,42,50,60
4,25,42,20,0,24,56,62,27,53
5,60,48,50,24,0,28,38,46,52
6,32,25,37,56,28,0,60,49,24
7,30,38,42,62,38,60,0,64,49
8,70,50,50,27,46,49,64,0,50
9,40,26,60,53,52,24,49,50,0


In [189]:
# cost matrix

distances = np.asarray(dist)
distances

array([[ 0, 45, 64, 25, 60, 32, 30, 70, 40],
       [45,  0, 30, 42, 48, 25, 38, 50, 26],
       [64, 30,  0, 20, 50, 37, 42, 50, 60],
       [25, 42, 20,  0, 24, 56, 62, 27, 53],
       [60, 48, 50, 24,  0, 28, 38, 46, 52],
       [32, 25, 37, 56, 28,  0, 60, 49, 24],
       [30, 38, 42, 62, 38, 60,  0, 64, 49],
       [70, 50, 50, 27, 46, 49, 64,  0, 50],
       [40, 26, 60, 53, 52, 24, 49, 50,  0]])

In [190]:
prob = gb.Model("TSP")
n = 9

In [191]:
U = prob.addVars(n, vtype = GRB.INTEGER,  lb = 1, ub =8, name = [ f"U{i+1}" for i in range(n)])
X = prob.addVars(n,n, vtype = GRB.BINARY, name = [f"X{i+1}-{j+1}" for i in range(n) for j in range(n)])

In [192]:
prob.setObjective(sum(distances[i][j]*X[i,j] for i in range(n) for j in range(n)), GRB.MINIMIZE)

In [193]:
for i in range(1,n):
    for j in range(1,n):
        if(i!=j):
            prob.addConstr(U[i]-U[j] + n*X[i,j] <= n-1)


prob.addConstrs(sum(X[i,j] for i in range(n)) == 1 for j in range(n) )
prob.addConstrs(sum(X[i,j] for j in range(n)) == 1 for i in range(n) )

prob.addConstrs(X[i,i] == 0 for i in range(n))
prob.addConstrs( U[i] <= n-1 for i in range(1,n))
prob.addConstrs( U[i] >= 1 for i in range(1,n))


{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>}

In [194]:
prob.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 99 rows, 90 columns and 355 nonzeros
Model fingerprint: 0x6224a432
Variable types: 0 continuous, 90 integer (81 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [2e+01, 7e+01]
  Bounds range     [1e+00, 8e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 404.0000000
Presolve removed 25 rows and 10 columns
Presolve time: 0.00s
Presolved: 74 rows, 80 columns, 312 nonzeros
Variable types: 0 continuous, 80 integer (72 binary)

Root relaxation: objective 2.610000e+02, 29 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  261.00000    0   14  404.00000  261.00000  35.4%     -    0s
H    0     0    

In [195]:
for v in prob.getVars():
    if v.x > 0:
        print(v.varName, "=", v.x)

U1 = 1.0
U2 = 6.0
U3 = 5.0
U4 = 4.0
U5 = 2.0
U6 = 8.0
U7 = 1.0
U8 = 3.0
U9 = 7.0
X1-7 = 1.0
X2-9 = 1.0
X3-2 = 1.0
X4-3 = 1.0
X5-8 = 1.0
X6-1 = 1.0
X7-5 = 1.0
X8-4 = 1.0
X9-6 = 1.0


In [196]:
prob.objVal

273.0